# Berkeley Hills earthquakes

Playing around with a dataset of earthquakes in the Berkeley Hills, for the research workflow workshop (Jupyter session), organized for the incoming graduate student cohort of the Earth and Planetary Science Department at the University of California, Berkeley.

## What is this?

> The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, data visualization, machine learning, and much more.

Basically, you can think of the notebook as an "all-inclusive" interactive document that integrates code, code output, and descriptive text. This is (almost) perfect for reproducible science!

* browser-based
* many languages supported (Python, R, Matlab... this notebook is in Python 3)

The core of any notebook is built upon two types of cells: **text** and **code**.

---

This is an example of a text cell.

You can write text here using Markdown syntax, and the notebook will typeset your text once you "execute" the cell.

Some examples...

#### look at me I'm a level 4 header

*look at me I'm italic text*

* I'm
* a
* list

> a quote, for when you can't be bothered to come up with your own words

`beep() boop.beep boop[]`

$$
\sum\frac{x^{2}}{y_{2}}
$$

~~why even bother showing me if you're going to cross me out anyway~~

| some other stuff | and more stuff |
|------------------|----------------|
| $\sigma$         | $\frac{X}{Y}$  |
| **bold**         | $\Sigma$       |

<font color='red'>RED TEXT</font>

Refer to websites such as this to learn more about Markdown syntax: https://www.markdownguide.org/cheat-sheet/

Refer to websites such as this to learn more about LaTeX syntax: https://wch.github.io/latexsheet/

---

Below is an example of a code cell:

In [ ]:
x = 1
y = 2

print(x + y)

---

You can perform notebook actions using the menubar, but here are a few actions that are very common so I find it just good to know the shortcuts to speed things up:

* command mode (`Esc`):
    * `a`: create code cell above
    * `b`: create code cell below
    * `d, d`: delete cell
    * `m`: convert cell into Markdown
    * `y`: convert cell into code
    
* edit mode (`Enter`):
    * `Shift + Enter`: execute cell and move one cell down
        * creates cell below if none exists
        
Other shortcuts can be viewed/edited in the `Help` menu.

## Let's do something

in Python. Sorry R/Matlab users.

### Importing the data

The dataset we will be playing around with contains information about earthquakes in the Berkeley Hills (https://earthquake.usgs.gov/earthquakes/search/). Fortunately for us, the dataset is already cleaned up and formatted as a nice comma-separated table, so let's read it in using `pandas`.

First, import standard modules:

In [ ]:
# pandas deals with data tables nicely
import pandas as pd

# matplotlib deals with plotting nicely
import matplotlib.pyplot as plt

# numpy deals with matrices nicely
import numpy as np

Read in the file using `pandas`:

In [ ]:
# read in the file
EQ = pd.read_csv('Berkeley-Hills-earthquakes.csv')

# display the top of the table to check if it was read in correctly
EQ.head()

In [ ]:
EQ['time'][0]

`pandas` was smart enough to read in most columns correctly, but it didn't do so well with the columns that contain time information. Let's fix that:

In [ ]:
# overwrite the raw columns with the correctly interpreted columns
EQ['time'] = pd.to_datetime(EQ['time'])
EQ['updated'] = pd.to_datetime(EQ['updated'])

EQ.head()

In [ ]:
EQ['time'][0]

### Data exploration

Let's get a rough sense of what we're dealing with:

In [ ]:
# general information about the dataframe
EQ.info()

In [ ]:
# basic stats for the numerical columns
EQ.describe()

In [ ]:
# quick plot histograms for the numerical columns
EQ.hist()

plt.show()

Doesn't look great... what can we do to make this look nicer?

* `Shift + Tab + Tab`: show docstring
* Google, stackoverflow, YouTube, your friends...

![reddit 1](image1.jpg)

![reddit 2](image2.jpg)

In [ ]:
EQ.hist(figsize=(15,10))

plt.show()

### Plotting

The quick and dirty way:

In [ ]:
# make the histogram
plt.hist(EQ['depth'])

# set the x and y labels
plt.xlabel('depth (km)')
plt.ylabel('n')

# show it
plt.show()

A far, far, far, superior way for the more intelligent among you:

(just kidding - the above way is fine)

In [ ]:
# set up the figure and axis handles
fig, ax = plt.subplots()

# make the histogram on the axis
ax.hist(EQ['depth'])

# x and y labels
ax.set_xlabel('depth (km)')
ax.set_ylabel('n')

# show it
plt.show(fig)

* more control
* less ambiguity
* more reproducible
* not(/less) sensitive to the order that you type things

In [ ]:
# set up the figure and axis handles
fig, ax = plt.subplots()

# make the histogram on the axis
ax.scatter(EQ['depth'], EQ['mag'])

# x and y labels
ax.set_xlabel('depth (km)')
ax.set_ylabel('magnitude')

# show it
plt.show(fig)

An example of getting a little fancy:

In [ ]:
# set up the figure and axis handles
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# make the basic plots
ax[0].hist(EQ['longitude'], color='C0')
ax[1].hist(EQ['latitude'], color='C1')
ax[2].scatter(EQ['longitude'], EQ['latitude'], c='C2')

# make the y-axis the same for the two histograms
hist_ylim_max = 120
ax[0].set_ylim(0, hist_ylim_max)
ax[1].set_ylim(0, hist_ylim_max)

# set axis labels
ax[0].set_xlabel('longitude')
ax[1].set_xlabel('latitude')
ax[2].set_xlabel('longitude')

# rotate x tick labels
for i in range(len(ax)):
    plt.setp(ax[i].get_xticklabels(), rotation=45)
    
# move the yaxis labels to the right for the scatter plot
ax[2].yaxis.tick_right()
ax[2].yaxis.set_label_position('right')
ax[2].set_ylabel('latitude', rotation=270, labelpad=20)

# label them
panel_labels = ['A', 'B', 'C']
for i in range(len(ax)):
    ax[i].text(0.95, 0.95, panel_labels[i],
               horizontalalignment='center', verticalalignment='center', transform=ax[i].transAxes)

plt.show(fig)

Isn't panel C a map?

Let's actually plot the data on a map. To do so, we'll use `cartopy`. To install this package:

```
conda install cartopy
```

or 

```
conda install -c conda-forge cartopy
```

In [ ]:
import cartopy.crs as ccrs

First, a global map:

In [ ]:
# set up the figure handle
fig = plt.figure(figsize=(15,6))

# add a Robinson projection axis to the figure
ax = plt.subplot(1,1,1, projection=ccrs.Robinson())

# draw the stock image
ax.stock_img()

# draw the coastlines
ax.coastlines()

# plot a point where Berkeley is
ax.scatter([-122.2730], [37.8715], s=100, c='C1', transform=ccrs.PlateCarree(), zorder=100)

plt.show()

Now zoom in to the area of interest, and plot the earthquake locations:

In [ ]:
# this function allows us to use other stock images
from cartopy.io import img_tiles

In [ ]:
# set up the figure handle
fig = plt.figure(figsize=(10,10))

# get the stock image
terrain = img_tiles.Stamen('terrain-background')

# add an axis to the figure, using the projection of the stock image
ax = plt.subplot(1,1,1, projection=terrain.crs)

# draw the stock image, at level 12 resolution
ax.add_image(terrain, 12)

# plot the earthquakes
ax.scatter(EQ['longitude'], EQ['latitude'], c='C1', s=10, transform=ccrs.Geodetic(), zorder=100)

# zoom to the area of interest
ax.set_extent([-122.35, -122.05, 37.750, 37.950], crs=ccrs.Geodetic())

# draw grid lines
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
             color='gray', alpha=0.5, linestyle='--')

plt.show()

### Restart and Run All

While developing code in the notebook, it is not uncommon to move code around, change variable names, paths to files, etc. This may unintentionally lead to a notebook that does not execute from start to finish (which defeats a large part of why these notebooks exist). It is therefore good practice to "Restart & Run All" to make sure your notebook actually executes.

## Other useful Jupyter things

* find and replace
    * `Edit` > `Find and Replace`
* table of contents
    * Jupyter notebook extensions: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/
```
conda install -c conda-forge jupyter_contrib_nbextensions
```
* progress bar
    * https://tqdm.github.io
* widgets
* other languages:
    * Python 2: https://docs.anaconda.com/anaconda/user-guide/tasks/switch-environment/
    * Matlab: https://am111.readthedocs.io/en/latest/jmatlab_install.html
    * R: https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/
* sharing:
    * as a static document:
        * `File` > `Download as`
        * https://nbviewer.jupyter.org
    * version-controlled sharing:
        * GitHub (learn more next week)
* Jupyter lab
    * instead of typing `jupyter notebook` in your terminal/command prompt, type `jupyter lab` instead

`tqdm`:

In [ ]:
# import the required modules
from tqdm import tqdm_notebook
from time import sleep

In [ ]:
# a simple nested for loop... but who knows how long we'll be waiting for?
for i in range(10):
    for j in range(100):
        sleep(0.01)

In [ ]:
# the same nested for loop, but now with a progress bar
for i in tqdm_notebook(range(10), desc='1st loop'):
    for j in tqdm_notebook(range(100), desc='2nd loop', leave=False):
        sleep(0.01)

`widgets`:

In [ ]:
# import the widgets module
import ipywidgets as widgets

# use the "magic" to set the values for the interactive variables
@widgets.interact(frequency=(-5.,5.), color=['blue', 'red', 'green'], lineweight=(1., 10.))

# define a function that will create the plot, using parameters that you want to be interactive
def widget_plot(frequency=1., color='blue', lineweight=2., grid=True):
    """
    Plot an interactive sin plot using ipywidgets.
    
    Parameters
    ----------
    frequency : float, optional
        Frequency of sin wave.
        
    color : string, optional
        Color of curve - must be 'blue', 'red', or 'green'.
        
    lineweight : float, optional
        Weight of curve.
        
    grid : boolean, optional
        If True, plot grid.
    """
    # set up the x-array
    x = np.linspace(-1., +1., 100)
    
    # set up the figure
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    
    # calculate y
    y = np.sin(2 * np.pi * frequency * x)
    
    # plot
    ax.plot(x, y, lw=lineweight, color=color)
    ax.grid(grid)

Note that, if formatted properly, your docstring will automatically be recognized as such:

## Other useful non-Jupyter things

* Atom
    * a text editor
    * https://atom.io
* Lepton
    * a code snippet manager
    * http://hackjutsu.com/Lepton/